# Two ways to open myplaces.kml (Google Earth) in Geopandas
Do you want to get your placesmarks from Google Earth into a geopandas GeoDataFrame? Well, you can't just call gpd.read_file() on a KML like you are used to, but it is possible if you register a fiona driver. However you'll be dissappointed if you organized your placemarks in folders and you do not want to loose this information ...

I propose an alternative way (without any additional packages) using python's minidom to get a GeoDataFrame including a column with the complete path of each placemark. KML is XML anyway ...

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from xml.dom.minidom import *
import fiona 

# Path to your myplaces.kml
url = "myplaces.kml"

## Open KML directly with geopandas and fiona driver
This is the easy and fast approach. 

In [ ]:

# Register driver
gpd.io.file.fiona.drvsupport.supported_drivers['LIBKML'] = 'rw'

# Open KML
gdf = gpd.read_file(url, driver='LIBKML')

# However the first approach might not work:
gdf.head()

With the first approach however, we only get the placemarks of the base folder of myplaces. Fiona treats each folder as layer and we can iterate through these layers:

In [ ]:
fiona.listlayers(url)

In [ ]:
gdf_list = []
for layer in fiona.listlayers(url):
    gdf  = gpd.read_file(url, driver='LIBKML', layer=layer)
    gdf['folder'] = layer
    gdf_list.append(gdf)

gdf = gpd.GeoDataFrame(pd.concat(gdf_list, ignore_index=True))
gdf.head()

Not bad, we even added a column with the folder of the placemark. However with if you have nested folders, you loose information of the parent folders. I tried several third party python KML libraries, but it turned out easier to parse the KML myself using pythons minidom. 
## Opening KML with minidom
Minidom is one of pythons standard ways to parse XML files. I define a function to recursively get all the parent folders of each folder recursively and then I get the data for each placemark. This code is slow, but good enough for personal use.

In [ ]:
dom = parse(url)

In [ ]:
# Get Path with all subfolders
def subfolders(node):
    if node.parentNode == dom.documentElement:
        return ""
    else:
        foldername = node.getElementsByTagName("name")[0].firstChild.data
        path = subfolders(node.parentNode) + "/" + foldername
    return path

In [ ]:
entries = []
placemarks = dom.getElementsByTagName("Placemark")

for i in placemarks:
    longitude = i.getElementsByTagName("longitude")[0].firstChild.data
    latitude = i.getElementsByTagName("latitude")[0].firstChild.data
    try:
        name = i.getElementsByTagName("name")[0].firstChild.data
    except:
        name = ""
    parent = i.parentNode
    foldername = parent.getElementsByTagName("name")[0].firstChild.data
    path = subfolders(parent) 
    entries.append((name, latitude, longitude, foldername, path)) # List of tuples

In [ ]:
df = pd.DataFrame(entries, columns=('name', 'latitude', 'longitude', 'folder', 'path'))

In [ ]:
df.head()

In [ ]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude, crs="EPSG:4326"))


In [ ]:
gdf.head()

In [ ]:
#gdf.to_csv("myplaces.csv")

In [ ]:
#gdf = gpd.read_file("myplaces.csv")

## Plots just for fun

In [ ]:
natearth = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
natearth.plot(ax=ax, color="darkgrey", edgecolor="lightgrey")
gdf.plot(ax=ax, color="red", marker=".")

In [ ]:
allpoints = gdf.dissolve()

fig, ax = plt.subplots(figsize=(10,5))
natearth.plot(ax=ax, color="darkgrey", edgecolor="lightgrey")
gdf.plot(ax=ax, color="blue", marker=".")
allpoints.convex_hull.plot(ax=ax, color="none", edgecolor="red")
ax.set_axis_off()

In [ ]:
# fig.savefig("bounding-box.png")